In [ ]:
import albumentations as alb
import copy
import time
import os
import cv2 
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
augmentor = alb.Compose([alb.RandomCrop(width=640, height=360), 
                         alb.HorizontalFlip(p=0.5), 
                         alb.RandomBrightnessContrast(p=0.2),
                         alb.RandomGamma(p=0.2), 
                         alb.RGBShift(p=0.2), 
                         alb.VerticalFlip(p=0.5)], 
                       bbox_params=alb.BboxParams(format='yolo', 
                                                  label_fields=['class_labels'],
                                                  min_area=1024, 
                                                  min_visibility=0.3))

In [ ]:
img = cv2.imread(os.path.join('weeddata','orignal','images','weed8.jpg'))

In [ ]:
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [ ]:
plt.imshow(img)

In [ ]:
boxlist=[]
boxlist.clear()
bboxfile=open(os.path.join('weeddata','orignal','labels','weed8.txt'))
for line in bboxfile:
    boxlist.append(list(map(float,(line.split()))))
boxlist

In [ ]:
class_labels =[]
for col in boxlist:
    class_labels.append(col[0])
    del col[0]
    

In [ ]:
class_labels

In [ ]:
boxlist
#boxlist = np.array(boxlist)

In [ ]:
w = 1280
h = 720
coords=[]
coords.clear()
for ls in boxlist:
    x1 = int( float(float(ls[0]) * w ))
    y1 = int( float(float(ls[1]) * h ))
    xw = int( float(float(ls[2]) * w /2))
    yw = int( float(float(ls[3]) * h /2))
    
    #coords.append([(float(float(ls[0]) * w )-float(float(ls[2]) * w /2))/w,
    #               (float(float(ls[1]) * h )-float(float(ls[3]) * h /2))/h,
    #               (float(float(ls[0]) * w )+float(float(ls[2]) * w /2))/w,
    #               (float(float(ls[1]) * h )+float(float(ls[3]) * h /2))/h])
    
    coords.append([x1,y1,xw,yw])
    
    start_point = (x1 - xw, y1 - yw )
    end_point = (x1 + xw, y1 + yw )
    thickness =5
    cv2.rectangle(img,start_point,end_point,(250,250,0),thickness)
             
plt.imshow(img)

In [ ]:
coords

In [ ]:
augmented = augmentor(image=img, bboxes=boxlist, class_labels=class_labels)

In [ ]:
augmented['bboxes']

In [ ]:
transformed =augmented['bboxes']
len(transformed[0])

In [ ]:
for x in range(60):
    augmented = augmentor(image=img, bboxes=boxlist, class_labels=class_labels)
    print(augmented['class_labels'])
    cv2.imwrite(os.path.join("weeddata","augmented","images","myfile.jpg"),augmented['image'])
    file =open(os.path.join("weeddata","augmented","labels","myfile.txt"), "w")
    for i in transformed:
        label =0
        file.write(str(int(augmented['class_labels'][label])))
        file.write(" ")
        for n in i:
            file.write(str(n))
            file.write(" ")
        if label < 4 :
            file.write("\n")
        label += 1
    file.close()
    


In [ ]:
augmented['class_labels']

In [ ]:
#cv2.putText(augmented['image'],
#            augmented['class_labels'][0],(int((augmented['bboxes'][0][0])*640)*10,int((augmented['bboxes'][0][1])*320)*10),
#            cv2.FONT_HERSHEY_SIMPLEX,
#            1,
#            (0,250,250),
#            5)
plt.imshow(augmented['image'])

In [ ]:
plt.imshow(augmented['image'])

In [ ]:
for image in os.listdir(os.path.join("weeddata","orignal","images")):
    orignal_image = cv2.imread(os.path.join("weeddata","orignal","images",image))
    bbox_file=os.path.join("weeddata","orignal","labels", image.split(".")[0]+".txt")
    
    orignal_image = cv2.cvtColor(orignal_image, cv2.COLOR_BGR2RGB)
    
    org_bbox=[]
    class_labels =[]
    alb_bbox = []
    
    if os.path.exists(bbox_file):
        bbox_file =open(bbox_file,"r")
        for line in bbox_file:
            org_bbox.append(list(map(float,line.split())))
        bbox_file.close()
    else:
        continue
    
    alb_bbox=copy.deepcopy(org_bbox)
   
    for col in alb_bbox:
        class_labels.append(col[0])
        del col[0]
    
    w = 640
    h = 360
    coords=[]
    
    for ls in alb_bbox:
        x1 = float(float(ls[0]) * w )
        y1 = float(float(ls[1]) * h )
        xw = float(ls[2]) 
        yw = float(ls[3]) 
    
        #coords.append([(float(float(ls[0]) * w )-float(float(ls[2]) * w /2))/w,
        #               (float(float(ls[1]) * h )-float(float(ls[3]) * h /2))/h,
        #               (float(float(ls[0]) * w )+float(float(ls[2]) * w /2))/w,
        #               (float(float(ls[1]) * h )+float(float(ls[3]) * h /2))/h])
       
        
        #print(x1,y1,xw,yw)
        coords.append([((x1+xw)/2)/640,
                       ((y1+yw)/2)/360,
                       xw/640,
                       yw/360])
    try:
        for x in range(60):
            print(class_labels)
            augmented = augmentor(image=orignal_image, bboxes=alb_bbox, class_labels=class_labels)
            #print(augmented['class_labels'])
            cv2.imwrite(os.path.join("weeddata","augmented","images",f'{image.split(".")[0]}_{x}.jpg'), augmented['image'])
            txt_file = open(os.path.join("weeddata","augmented","labels",f'{image.split(".")[0]}_{x}.txt'),"w")
            
            for items in augmented['bboxes']:
                label = 0
                txt_file.write(str(int(augmented['class_labels'][label])))
                txt_file.write(" ")
                for n in items:
                    txt_file.write(str(n))
                    txt_file.write(" ")
                if label < len(augmented['bboxes']) :
                    txt_file.write("\n")
                label += 1
            
            txt_file.close()
                
    except Exception as e:
        print(e)
            

In [ ]:
org_bbox.clear()
    class_labels.clear()
    alb_bbox.clear()
    coords.clear()
class_labels